In [120]:
from FINData import FINData
import vectorbtpro as vbt
import numpy as np
from numba import njit
import pandas as pd 
data=FINData.pull(
    ['CEO'],
    start='2015-01-01'
)

In [121]:
price_window=data.hlc3

In [122]:
price_window.vbt.plot().show()

In [123]:
pattern = np.array([1, 2, 3, 2, 3, 2])
pd.Series(pattern).vbt.plot().show()

In [124]:
resized_pattern = vbt.nb.interp_resize_1d_nb(
    pattern, 90, vbt.enums.InterpMode.Mixed
)
resized_pattern

array([1.        , 1.05617978, 1.11235955, 1.16853933, 1.2247191 ,
       1.28089888, 1.33707865, 1.39325843, 1.4494382 , 1.50561798,
       1.56179775, 1.61797753, 1.6741573 , 1.73033708, 1.78651685,
       1.84269663, 1.8988764 , 1.95505618, 2.        , 2.06741573,
       2.12359551, 2.17977528, 2.23595506, 2.29213483, 2.34831461,
       2.40449438, 2.46067416, 2.51685393, 2.57303371, 2.62921348,
       2.68539326, 2.74157303, 2.79775281, 2.85393258, 2.91011236,
       2.96629213, 3.        , 2.92134831, 2.86516854, 2.80898876,
       2.75280899, 2.69662921, 2.64044944, 2.58426966, 2.52808989,
       2.47191011, 2.41573034, 2.35955056, 2.30337079, 2.24719101,
       2.19101124, 2.13483146, 2.07865169, 2.        , 2.03370787,
       2.08988764, 2.14606742, 2.20224719, 2.25842697, 2.31460674,
       2.37078652, 2.42696629, 2.48314607, 2.53932584, 2.59550562,
       2.65168539, 2.70786517, 2.76404494, 2.82022472, 2.87640449,
       2.93258427, 3.        , 2.95505618, 2.8988764 , 2.84269

In [125]:
pattern_scale = (resized_pattern.min(), resized_pattern.max())
price_window_scale = (price_window['CEO'].min(), price_window['CEO'].max())
rescaled_pattern = vbt.utils.array_.rescale_nb(
    resized_pattern, pattern_scale, price_window_scale
)


In [126]:
rescaled_pattern

array([ 4.20333333,  5.88985019,  7.57636704,  9.2628839 , 10.94940075,
       12.6359176 , 14.32243446, 16.00895131, 17.69546816, 19.38198502,
       21.06850187, 22.75501873, 24.44153558, 26.12805243, 27.81456929,
       29.50108614, 31.187603  , 32.87411985, 34.22333333, 36.24715356,
       37.93367041, 39.62018727, 41.30670412, 42.99322097, 44.67973783,
       46.36625468, 48.05277154, 49.73928839, 51.42580524, 53.1123221 ,
       54.79883895, 56.48535581, 58.17187266, 59.85838951, 61.54490637,
       63.23142322, 64.24333333, 61.88220974, 60.19569288, 58.50917603,
       56.82265918, 55.13614232, 53.44962547, 51.76310861, 50.07659176,
       48.39007491, 46.70355805, 45.0170412 , 43.33052434, 41.64400749,
       39.95749064, 38.27097378, 36.58445693, 34.22333333, 35.23524345,
       36.9217603 , 38.60827715, 40.29479401, 41.98131086, 43.66782772,
       45.35434457, 47.04086142, 48.72737828, 50.41389513, 52.10041199,
       53.78692884, 55.47344569, 57.15996255, 58.8464794 , 60.53

In [127]:
price = price_window['CEO']  # Series giá HPG length ~ 2723
pattern = pattern_scale

In [128]:
similarity = price.vbt.rolling_pattern_similarity(
    pattern,
    window=90,              # mặc định = len(pattern)
    error_type="relative",    # dùng sai số tương đối
    max_error=0.05,           # cho phép lệch 5%
    max_error_interp_mode="discrete"
)

In [129]:
top_matches = similarity.nlargest(5)
print(top_matches)

DateTime
2017-01-23 00:00:00+00:00    0.912041
2017-01-20 00:00:00+00:00    0.912033
2017-01-24 00:00:00+00:00    0.910902
2017-01-19 00:00:00+00:00    0.910315
2017-02-06 00:00:00+00:00    0.909261
Name: CEO, dtype: float64


In [130]:
win_len = len(pattern)   # hoặc window bạn dùng trong rolling_pattern_similarity

for ts, sim in top_matches.items():
    # Lấy vị trí (integer index) tương ứng với timestamp ts
    end_pos = price.index.get_loc(ts)   # ví dụ 1234

    # Tính start_pos theo số bar
    start_pos = max(0, end_pos - win_len + 1)

    window_price = price.iloc[start_pos:end_pos + 1]

    print(f"\nPattern match tại {ts} (similarity={sim:.3f}) "
          f"→ dùng window [{start_pos}:{end_pos}] ({len(window_price)} bar)")

    fig = window_price.vbt.plot()
    fig.show()



Pattern match tại 2017-01-23 00:00:00+00:00 (similarity=0.912) → dùng window [512:513] (2 bar)



Pattern match tại 2017-01-20 00:00:00+00:00 (similarity=0.912) → dùng window [511:512] (2 bar)



Pattern match tại 2017-01-24 00:00:00+00:00 (similarity=0.911) → dùng window [513:514] (2 bar)



Pattern match tại 2017-01-19 00:00:00+00:00 (similarity=0.910) → dùng window [510:511] (2 bar)



Pattern match tại 2017-02-06 00:00:00+00:00 (similarity=0.909) → dùng window [517:518] (2 bar)


In [132]:
price = price_window['CEO'] 
pattern = np.array([1, 2, 3, 2, 3, 2])
similarity = price.vbt.rolling_pattern_similarity(
    pattern, 
    window=30,
    error_type="relative",
    max_error=0.05,
    max_error_interp_mode="discrete"
)
similarity.describe()

count    2696.000000
mean        0.501931
std         0.132102
min         0.166642
25%         0.404948
50%         0.504128
75%         0.592710
max         0.847977
Name: CEO, dtype: float64

In [133]:
similarity.argmax()

58

In [134]:
end_row = similarity.argmax() + 1  
start_row = end_row - 30
fig = data.iloc[start_row:end_row].plot(plot_volume=False)
price.iloc[start_row:end_row].vbt.plot_pattern(
    pattern, 
    error_type="relative",  
    max_error=0.05,
    max_error_interp_mode="discrete",
    plot_obj=False, 
    fig=fig
)
fig.show()

In [135]:
end_row = similarity.argmin() + 1
start_row = end_row - 30
fig = data.iloc[start_row:end_row].plot(plot_volume=False)
price.iloc[start_row:end_row].vbt.plot_pattern(
    pattern, 
    invert=True,
    error_type="relative",
    max_error=0.05,
    max_error_interp_mode="discrete",
    plot_obj=False, 
    fig=fig
)
fig.show()

In [136]:
inv_similarity = price.vbt.rolling_pattern_similarity(
    pattern, 
    window=30,
    invert=True,  
    error_type="relative",
    max_error=0.05,
    max_error_interp_mode="discrete"
)
end_row = inv_similarity.argmax() + 1  
start_row = end_row - 30
fig = data.iloc[start_row:end_row].plot(plot_volume=False)
price.iloc[start_row:end_row].vbt.plot_pattern(
    pattern, 
    invert=True,
    error_type="relative",
    max_error=0.05,
    max_error_interp_mode="discrete",
    plot_obj=False, 
    fig=fig
)
fig.show()

In [137]:
patsim = vbt.PATSIM.run(
    price, 
    vbt.Default(pattern),  
    error_type=vbt.Default("relative"),
    max_error=vbt.Default(0.05),
    max_error_interp_mode=vbt.Default("discrete"),
    window=[30, 45, 60, 75, 90]
)

In [138]:
patsim.wrapper.columns

Index([30, 45, 60, 75, 90], dtype='int64', name='patsim_window')

In [139]:
patsim.plot(column=60).show()

In [140]:
patsim.overlay_with_heatmap(column=60).show()

In [141]:
exits = patsim.similarity >= 0.8
exits.sum()

patsim_window
30    15
45    17
60    14
75     5
90    13
dtype: int64

In [142]:
patsim.similarity

patsim_window,30,45,60,75,90
DateTime,,,,,
2015-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2015-01-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2015-01-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2015-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2015-01-09 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-11-24 00:00:00+00:00,0.232707,0.501241,0.412865,0.400906,0.584810
2025-11-25 00:00:00+00:00,0.231578,0.478599,0.419667,0.386061,0.576504
2025-11-26 00:00:00+00:00,0.236273,0.472571,0.432912,0.376781,0.572445


In [143]:
patsim = vbt.PATSIM.run(
    price, 
    vbt.Default(pattern),
    error_type=vbt.Default("relative"),
    max_error=vbt.Default(0.05),
    max_error_interp_mode=vbt.Default("discrete"),
    window=[30, 45, 60, 75, 90],
    invert=[False, True],
    min_similarity=[0.7, 0.8],
    param_product=True  
)
exits = ~patsim.similarity.isnull()  
exits.sum()

patsim_window  patsim_invert  patsim_min_similarity
30             False          0.7                      182
                              0.8                       15
               True           0.7                      292
                              0.8                       32
45             False          0.7                      204
                              0.8                       17
               True           0.7                      305
                              0.8                       26
60             False          0.7                      231
                              0.8                       14
               True           0.7                      328
                              0.8                       21
75             False          0.7                      235
                              0.8                        5
               True           0.7                      313
                              0.8                       24
90  

In [144]:
groupby = [  
    name for name in patsim.wrapper.columns.names 
    if name != "patsim_window"
]
max_sim = patsim.similarity.groupby(groupby, axis=1).max()  
entries = ~max_sim.xs(True, level="patsim_invert", axis=1).isnull()  
exits = ~max_sim.xs(False, level="patsim_invert", axis=1).isnull()

C:\Users\hung.hm\AppData\Local\Temp\ipykernel_944\721666788.py:5: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



In [145]:
pattern_range_records = vbt.nb.find_pattern_1d_nb(
    price.values,  
    pattern,
    window=30,  
    max_window=90,
    error_type=vbt.enums.ErrorType.Relative,
    max_error=np.array([0.05]),  
    max_error_interp_mode=vbt.enums.InterpMode.Discrete,
    min_similarity=0.85
)
pattern_range_records

array([(0, 0,   28,   60, 1, 0.85496689),
       (1, 0,  457,  488, 1, 0.850009  ),
       (2, 0,  796,  860, 1, 0.85699163),
       (3, 0, 1660, 1698, 1, 0.86436987),
       (4, 0, 2395, 2438, 1, 0.85169886)],
      dtype={'names': ['id', 'col', 'start_idx', 'end_idx', 'status', 'similarity'], 'formats': ['<i8', '<i8', '<i8', '<i8', '<i8', '<f8'], 'offsets': [0, 8, 16, 24, 32, 40], 'itemsize': 48, 'aligned': True})

In [146]:
start_row = pattern_range_records[1]["start_idx"]
end_row = pattern_range_records[1]["end_idx"]
fig = data.iloc[start_row:end_row + 30].plot(plot_volume=False)
price.iloc[start_row:end_row].vbt.plot_pattern(
    pattern, 
    error_type="relative",
    max_error=0.05,
    max_error_interp_mode="discrete",
    plot_obj=False, 
    fig=fig
)
fig.show()

In [147]:
pattern_ranges = vbt.PatternRanges.from_pattern_search(
    price,  
    pattern,
    window=30,
    max_window=90,
    error_type="relative",  
    max_error=0.05,  
    max_error_interp_mode="discrete",
    min_similarity=0.85
)
pattern_ranges

In [148]:
pattern_ranges = price.vbt.find_pattern(
    pattern,
    window=30,
    max_window=90,
    error_type="relative",
    max_error=0.05,
    max_error_interp_mode="discrete",
    min_similarity=0.85
)

In [152]:
pattern_ranges.plot(width=1280,height=860).show()

In [ ]:
pattern_ranges.loc["2021-09-01":"2022-01-01"].plot().show()